In [1]:
#ipython magic
%reset -f
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
#general imports
import sys
sys.path.append('..')
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [3]:
#notebook specific imports
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from utils import gsheets_importer as gsheet 
from paq2py import paq_read
from utils.opto_stim_import2 import OptoStim1p, OptoStim2p 
from utils.utils_funcs import paq_data
import fnmatch
from utils.rsync_aligner import Rsync_aligner


In [4]:
#global plotting params
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
sns.set()
sns.set_style('white')

In [5]:
#### spreadsheet settings

#from the URL
sheet_ID = '1GG5Y0yCEw_h5dMfHBnBRTh5NXgF6NqC_VVGfSpCHroc'

#the name of the file path column
date_header = 'Date'
t_series_header = 't-series name'
paq_header = '.paq file'
naparm_header = 'Naparm'
blimp_header = 'blimp folder'
pycontrol_header = 'pycontrol txt'

#column telling whether to analyse or not
analyse_bool_header = 'Analyse'


In [6]:
#the sheet is named after the mouse in my spreadsheet
mouse_name = 'RL022'
#have to give it rows and columns, so just given a huge range here
SHEET_NAME = mouse_name + '!A1:Z69'

df = gsheet.gsheet2df(sheet_ID, HEADER_ROW=2, SHEET_NAME=SHEET_NAME)
#comment this out if not using behaviour spreadsheet
df = gsheet.correct_behaviour_df(df)
df

,Date,Trained 1P,Trained 2P,Total rewards,Run Number,pycontrol txt,pycontrol task,Analyse,Comments,Start Autoreward,...,Spirals,Repeats,inter group interval (ms),Power / cell (mW),blimp folder,Naparm,.paq file,t-series name,Number of frames,Suite2p me?
0,2019-04-24,TRUE,FALSE,0,1,RL022-2019-04-24-164519,opto_stim,FALSE,Mouse not interested in licking,TRUE,...,,,,,,,,,,
1,2019-04-25,TRUE,FALSE,45,2,RL022-2019-04-25-122010,opto_stim,TRUE,mouse drank autoreward but did not lick before...,TRUE,...,,,,,,,,,,
2,2019-04-26,TRUE,FALSE,107,3,RL022-2019-04-26-180119,opto_stim,TRUE,"Mouse did well, understood task",TRUE,...,,,,,,,,,,
3,2019-04-27,TRUE,FALSE,69,4,RL022-2019-04-27-100459,opto_stim,TRUE,Mouse completed task to lowest power level aft...,FALSE,...,,,,,,,,,,
4,2019-04-30,FALSE,TRUE,90,5,RL024-2019-04-30-202257,opto_stim_scope,TRUE,WARNING DATA IS ERROREOUSLY RECORDED AS RL024....,NA,...,1x10ms,10,5,6,2019-04-30-202235,2019-04-30_RL024_NAPARM_002,2019-04-30_RL024_002,None,None,FALSE
5,2019-04-30,FALSE,TRUE,90,6,RL024-2019-04-30-212604,opto_stim_scope,TRUE,Turned the laser off but otherwise exactly the...,NA,...,1x10ms,10,5,0,2019-04-30-202235,2019-04-30_RL024_NAPARM_002,2019-04-30_RL024_002,None,None,FALSE
6,2019-05-01,FALSE,TRUE,122,7,RL022-2019-05-01-195448,opto_stim_scope,TRUE,Mouse did excellent (possibility paq is wrong),NA,...,1x10ms,10,5,6,2019-05-01-195339,2019-05-01_RL022_NAPARM_002,2019-05-01_RL022_003.paq,None,None,FALSE
7,2019-05-01,FALSE,TRUE,122,8,RL022-2019-05-01-210056,opto_stim_scope,TRUE,Turned off laser. Mouse performed at chance. T...,NA,...,1x10ms,10,5,0,2019-05-01-210033,2019-05-01_RL022_NAPARM_002,2019-05-01_RL022_003.paq,None,None,FALSE


In [7]:
def intersect(A, B):
    '''returns a set of the common items between two lists'''
    return set(A) - (set(A) - set(B))

In [8]:
idx_analyse = gsheet.df_bool(df, analyse_bool_header)
idx_2p = gsheet.df_bool(df, 'Trained 2P')
idx_1p = gsheet.df_bool(df, 'Trained 1P')

rows_2p = intersect(idx_analyse, idx_2p)
rows_1p = intersect(idx_analyse, idx_1p)

dates_2p = gsheet.df_col(df, date_header, rows_2p)
paqs = gsheet.df_col(df, paq_header, rows_2p)
naparm_folders = gsheet.df_col(df, naparm_header, rows_2p)
blimp_folders = gsheet.df_col(df, blimp_header, rows_2p)
pycontrol_folders = gsheet.df_col(df, pycontrol_header, rows_2p)


assert len(paqs) == len(naparm_folders) == len(blimp_folders) == len(dates_2p)

num_runs = len(paqs)

In [9]:
packerstation = '/home/jamesrowland/Documents/packerstation/jrowland/Data'

for run in [0]:#range(num_runs):
    
    date = dates_2p[run]
    paq = paqs[run]
    naparm = naparm_folders[run]
    blimp = blimp_folders[run]
    pycontrol = pycontrol_folders[run]
    
    umbrella = os.path.join(packerstation, date)
    blimp_path, naparm_path = gsheet.path_finder(umbrella, blimp, naparm, is_folder=True)
    pycontrol_path, paq_path = gsheet.path_finder(umbrella, pycontrol, paq, is_folder=False)
    
    with open(os.path.join(blimp_path, 'blimpAlignment.txt'), 'r') as f:
        barcode = [float(line.split(' ')[7][:-1]) for line in f.readlines()] #take the full stop off the end
            
    #build behaviour object from the pycontrol txt file
    behav = OptoStim2p(pycontrol_path)
    
    #test that the list of barcodes printed in the pycontrol sequence is contained in the 
    #list of barcodes from the alignment folder
    if not ''.join([str(b) for b in behav.barcode]) in ''.join([str(b) for b in barcode]):
        raise ValueError('pycontrol {} does not match blimp folder {}'.format(pycontrol, blimp))
    else:
        print('pycontrol {} successfully matched to blimp folder {}'.format(pycontrol, blimp))
        
    paq_obj = paq_read(paq_path)
    
    paq_rsync = paq_data(paq_obj, 'pycontrol_rsync', threshold_ttl=True, plot=False)
    
    try:
        aligner = Rsync_aligner(pulse_times_A=behav.rsync, pulse_times_B=paq_rsync,
                                units_B=1000/paq_obj['rate'],  plot=False, raise_exception=True)
        
        print('pycontrol {} rsync successfully matched to paq {}'.format(pycontrol, paq))
    except:
        raise ValueError('pycontrol {} rsync could not be matched to paq {}'.format(pycontrol, paq))

        
        
        
            

    


Importing data file: RL024-2019-04-30-202257.txt
pycontrol RL024-2019-04-30-202257 successfully matched to blimp folder 2019-04-30-202235
pycontrol RL024-2019-04-30-202257 rsync successfully matched to paq 2019-04-30_RL024_002


AttributeError: 'BlimpImport' object has no attribute 'parse_spreadsheet'